# Import libraries

In [ ]:
import pandas as pd
import json
from collections import defaultdict
from bs4 import BeautifulSoup
import requests
import os
import re
from collections import Counter
from functools import reduce
from tqdm.notebook import tqdm
from functools import reduce
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import heapq
from itertools import product
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem import *

# Import custom functions

In [ ]:
import custom_functions

# Import functions

#### In functions.py there's a series of functions that we have used; this choice was made to enhance readability

In [ ]:
import functions

# 1 Data collection

## [1.1] Get the list of master's degree courses

#### We create a .txt file (msc_urls.txt) whose single line corresponds to the master's URL.

In [ ]:
# Output file path
output_file_path = 'msc_urls.txt'
# Loop through the first 400 pages and write results to the output file
with open(output_file_path, 'a') as output_file:
    for page_number in range(1, 401):
        page_url = f'https://www.findamasters.com/masters-degrees/msc-degrees/?PG={page_number}'
        page_results = functions.extract_masters(page_url)
        for url in page_results:
            output_file.write(f'{url}\n')
        time.sleep(1)

#### Each page has 15 courses, so you will end up with 6000 unique master's degree URLs. Let's check it!

In [ ]:
with open(output_file_path, 'r') as file:
    lines = file.readlines()
    number_of_lines = len(lines)

print(f'The file {output_file_path} contains {number_of_lines} rows.')

The file msc_urls.txt contains 6000 rows.


## [1.2] Crawl master's degree pages

In [ ]:
# Function to create a directory if it doesn't exist
def create_directory(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

# Main directory for downloaded HTML files
main_directory = 'downloaded_pages'
create_directory(main_directory)

# Iterate through the URLs and download the HTML
with open('msc_urls.txt', 'r') as file:
    for index, url in enumerate(file, start=1):
        # Remove whitespaces and newline characters from the URL
        url = url.strip()

        # Generate the directory path for the current page
        page_directory = os.path.join(main_directory, f'page_{(index - 1) // 15 + 1}')
        create_directory(page_directory)

        # Generate the output HTML file path
        output_path = os.path.join(page_directory, f'course_{index}.html')

        # Download the HTML and save it to the file
        custom_functions.crawler(url, output_path)
        time.sleep(3)

print("Download complete for all pages.")

Download complete for all pages.


#### Now we have a folder named 'downloaded_pages' divided into 400 folders (one for each page), and each folder contains the HTML of the 15 courses present on that page.

## [1.3] Parse downloaded pages


In [ ]:
custom_functions.parser('downloaded_pages/page_1/course_1.html')

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,administration,country,url
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,3D visualisation and animation play a role in ...,September,Please see the university website for further ...,MSc,1 year full-time,Glasgow,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...


## Applying the function to our html pages

we test our parsing functioni to a single document and the ìn collect all information with the parser thoìrough the html pages

In [ ]:
concatenated_df=functions.parse_html('downloaded_pages')

Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_118/course_1765.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_119/course_1772.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_128/course_1910.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_140/course_2086.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_196/course_2929.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_196/course_2931.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_215/course_3213.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_291/course_4357.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_293/course_4395.html
Error parsing file: /content/drive/MyDrive/HM3-ADM/HW3_ADM/downloaded_pages/page_2

#### At this point, you should have all the HTML documents about the master's degree of interest, and you can start to extract specific information. The list of the information we desire for each course and their format as desiried.

In [ ]:
# Display the concatenated DataFrame
concatenated_df.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,administration,country,url
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,3D visualisation and animation play a role in ...,September,Please see the university website for further ...,MSc,1 year full-time,Glasgow,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
1,Air Quality Solutions - MSc,University of Leeds,Institute for Transport Studies,Full time,Up to 7 million people are estimated to die ev...,September,"UK: £12,500 (Total) \nInternational: £28,750 (...",MSc,"1 year full time, 2 or 3 years part-time",Leeds,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2,Analytical Toxicology MSc,King’s College London,Faculty of Life Sciences & Medicine,Full time,The Analytical Toxicology MSc is a unique stud...,See Course,Please see the university website for further ...,MSc,Full-time: One year,London,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
3,Applied Computer Science and Artificial Inte...,University of Bradford,Faculty of Engineering & Digital Technologies,Full time,Computer science is the foundation of many exc...,"September, January",Please see the university website for further ...,MSc,1 Year Full Time / 2 Years Part Time,Bradford,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
4,Applied Economics (Banking and Financial Mar...,University of Bath,University of Bath Online,Part time,From political uncertainty to finance and recr...,"September, January",Cost per 10 credits £722* (10% alumni discount...,MSc,2 years and 6 months full time,Bath,Online,United Kingdom,https://www.findamasters.com/masters-degrees/c...


In [ ]:
concatenated_df.shape

(5979, 13)

In [ ]:
print(6000-5979, 'were not valid pages')

21 were not valid pages


are all categorical variables

In [ ]:
parsed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   courseName      1 non-null      object
 1   universityName  1 non-null      object
 2   facultyName     1 non-null      object
 3   isItFullTime    1 non-null      object
 4   description     1 non-null      object
 5   startDate       1 non-null      object
 6   fees            1 non-null      object
 7   modality        1 non-null      object
 8   duration        1 non-null      object
 9   city            1 non-null      object
 10  administration  1 non-null      object
 11  country         1 non-null      object
 12  url             1 non-null      object
dtypes: object(13)
memory usage: 232.0+ bytes


In [ ]:
# Specify the path where you want to save the .tsv file
tsv_file_path = 'MasterDegrees.tsv'

# Save the DataFrame to a .tsv file
concatenated_df.to_csv(tsv_file_path, sep='\t', index=False)

print(f".tsv file saved at: {tsv_file_path}")

.tsv file saved at: /content/drive/MyDrive/HM3-ADM/HW3_ADM/MasterDegrees.tsv


### *Saving singularly the information about each master (html page)*

In [ ]:
# Iterate through each DataFrame in the list
for index, parsed_df in enumerate(parsed_dfs):
    # Iterate through each row in the DataFrame
    for row_index, row in parsed_df.iterrows():
        # Replace NaN values with empty strings
        single_row = row.fillna(' ')
        # Specify the path where you want to save the .tsv file for the current row
        tsv_file_path = f'Courses/course_{index}.tsv'
        # Save the single row DataFrame to a .tsv file
        with open(tsv_file_path, 'w') as file:
            file.write('\t'.join(single_row))

#[2.0]

## Preproccessing

Uploading the file created after parsing the information

In [ ]:
df = pd.read_csv('MasterDegrees.tsv',sep='\t')
df.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,administration,country,url
0,Air Quality Solutions - MSc,University of Leeds,Institute for Transport Studies,Full time,Up to 7 million people are estimated to die ev...,September,"UK: £12,500 (Total) \nInternational: £28,750 (...",MSc,"1 year full time, 2 or 3 years part-time",Leeds,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
1,Analytical Toxicology MSc,King’s College London,Faculty of Life Sciences & Medicine,Full time,The Analytical Toxicology MSc is a unique stud...,See Course,Please see the university website for further ...,MSc,Full-time: One year,London,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2,Applied Computer Science and Artificial Inte...,University of Bradford,Faculty of Engineering & Digital Technologies,Full time,Computer science is the foundation of many exc...,"September, January",Please see the university website for further ...,MSc,1 Year Full Time / 2 Years Part Time,Bradford,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
3,Applied Economics (Banking and Financial Mar...,University of Bath,University of Bath Online,Part time,From political uncertainty to finance and recr...,"September, January",Cost per 10 credits £722* (10% alumni discount...,MSc,2 years and 6 months full time,Bath,Online,United Kingdom,https://www.findamasters.com/masters-degrees/c...
4,Applied Linguistics - MSc,University of Glasgow,College of Arts & Humanities,Full time,This Masters focuses on how linguistic researc...,September,Please see the university website for further ...,MSc,1 year full-time; 2 years part-time,Glasgow,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...


## [2.0.0]

In [ ]:
# Download the stopwords dataset if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Stemming

In [ ]:
# Make a copy of the original DataFrame
processed_df = df.copy()

# stemmer
stemmer = PorterStemmer()

processed_df['description'] = processed_df.description.apply(lambda row: ' '.join([stemmer.stem(word) for word in row.split(' ')]))

## Lower, removing punctuation and stepwords

In [ ]:
stop_words = set(stopwords.words('english'))

# Selecting columns
text_columns = processed_df.select_dtypes(include='object').columns
# Lower
processed_df[text_columns] = processed_df[text_columns].apply(lambda x: x.str.lower() if x.dtype == 'O' else x)

# Exclude fees from removing punctuation
exclude_column = 'fees'

# Applying clean function to all the valid columns except the excluded one
processed_df[text_columns.difference([exclude_column])] = processed_df[text_columns.difference([exclude_column])].applymap(functions.clean)

we verify which element we're filtering to check and think if something else need to be considered. Checking buy some outuput of the stemmed words we see some additional punctuation we don't want to consider which was added within the clean function.

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

We can know appreciate the differences of the two datasets (before and after preprocessing). Of our interesting will be the columns **fees** and **description**.

In [ ]:
processed_df.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,administration,country,url
0,air quality solutions msc,university leeds,institute transport studies,full time,7 million peopl estim die everi year due harm ...,september,"uk: £12,500 (total) \ninternational: £28,750 (...",msc,1 year full time 2 3 years part-time,leeds,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...
1,analytical toxicology msc,king college london,faculty life sciences medicine,full time,analyt toxicolog msc uniqu studi cours integr ...,see course,please see the university website for further ...,msc,full-time one year,london,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...
2,applied computer science artificial intelligen...,university bradford,faculty engineering digital technologies,full time,comput scienc foundat mani excit cutting-edg t...,september january,please see the university website for further ...,msc,1 year full time 2 years part time,bradford,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...
3,applied economics banking financial markets on...,university bath,university bath online,part time,polit uncertainti financ recruit demands econo...,september january,cost per 10 credits £722* (10% alumni discount...,msc,2 years 6 months full time,bath,online,united kingdom,https //www.findamasters.com/masters-degrees/c...
4,applied linguistics msc,university glasgow,college arts humanities,full time,thi master focus linguist research help solv l...,september,please see the university website for further ...,msc,1 year full-time 2 years part-time,glasgow,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...


## [2.0.1]

 ## We want the field fees to collect numeric information


We consider only *fees* column and deepen what we find

In [ ]:
raw_fees= pd.DataFrame(processed_df['fees'])

In [ ]:
raw_fees.head()

,fees
0,"uk: £12,500 (total) \ninternational: £28,750 (..."
1,please see the university website for further ...
2,please see the university website for further ...
3,cost per 10 credits £722* (10% alumni discount...
4,please see the university website for further ...


there are lot of missings data or more complex information we need to filter. To filter we follow the given guidline:

>Really missing data (empty strings -122 of them), and something like this:
>>please see the university website for further information on fees for this course.

We decide to set this values to `none`


In [ ]:
# Missing values for fees
print(sum(raw_fees['fees'].isna()==True))

122



> in case of multiple information, retrieve only the highest fees. This suits perfectly the following example
>>*'uk £13,000 total international £29,000 total',
       '*

### Processing fees

Storing the rates in a file when needed to have an update versions of the rates.

In [ ]:
# Your ExchangeRate-API key
api_key = '40f223580924eaf7a1eb4ee0'

# Fetch exchange rates from the API for all currencies against USD
api_url = f'https://open.er-api.com/v6/latest/USD?apikey={api_key}'
response = requests.get(api_url)
data = response.json()
exchange_rates = data['rates']

# Define a mapping between currency symbols in your data and API symbols
currency_symbol_mapping = {
    '£': 'GBP',
    '€': 'EUR',
    '$': 'USD',
    '¥': 'JPY',
    'sek':'SEK', #swedish corona
    'euro':'EUR',
    'hkn' : 'HNK',
    'euros': 'EUR',
    'jpy': 'JPY',
    'hkd':'HDK',
    'isk':'ISK',
    'hkd$': 'HKD',
    'gbp£': 'GBP'
}

pattern = r'(?P<symbol_before>[£$€¥]|euros|eur|jpy|sek|euro|hkn|hkd|isk|hkd$|gbp£)?\s*(?P<value>\d{1,3}(?:,\d{3})*)\s*(?P<symbol_after>[£$€¥]|euros|jpy|sek|euro|hkn|hkd|isk|hkd$|gbp£)?'


apply the function which convert into a target value all entries of fees column

In [ ]:
text_columns = processed_df.select_dtypes(include='object').columns
# Return the list of currency symbols and costs encountered in 'fees' field through CONVERT TO COMMON CURRENCY function
raw_fees['fees (USD)'] = raw_fees['fees'].apply(lambda x: functions.return_cost(x) if x is not None else None)


`original` information and `filtered fees`

In [ ]:
raw_fees.head()

,fees,fees (USD)
0,"uk: £12,500 (total) \ninternational: £28,750 (...",35935.84
1,please see the university website for further ...,NaN
2,please see the university website for further ...,NaN
3,cost per 10 credits £722* (10% alumni discount...,902.46
4,please see the university website for further ...,NaN


substituing and renaming the new column with the appropriate currance values

In [ ]:
processed_df['fees']= raw_fees['fees (USD)']
processed_df.rename(columns={'fees': 'fees (USD)'}, inplace=True)


In [ ]:
processed_df.head()

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees (USD),modality,duration,city,administration,country,url,fees (USD)
0,air quality solutions msc,university leeds,institute transport studies,full time,7 million peopl estim die everi year due harm ...,september,35935.84,msc,1 year full time 2 3 years part-time,leeds,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...,35935.84
1,analytical toxicology msc,king college london,faculty life sciences medicine,full time,analyt toxicolog msc uniqu studi cours integr ...,see course,NaN,msc,full-time one year,london,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...,NaN
2,applied computer science artificial intelligen...,university bradford,faculty engineering digital technologies,full time,comput scienc foundat mani excit cutting-edg t...,september january,NaN,msc,1 year full time 2 years part time,bradford,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...,NaN
3,applied economics banking financial markets on...,university bath,university bath online,part time,polit uncertainti financ recruit demands econo...,september january,902.46,msc,2 years 6 months full time,bath,online,united kingdom,https //www.findamasters.com/masters-degrees/c...,902.46
4,applied linguistics msc,university glasgow,college arts humanities,full time,thi master focus linguist research help solv l...,september,NaN,msc,1 year full-time 2 years part-time,glasgow,campus,united kingdom,https //www.findamasters.com/masters-degrees/c...,NaN


## [2.1] Conjunctive query

## [2.1.1] Create your index!

### Create a file named vocabulary, in the format you prefer, that maps each word to an integer (id).

#### First of all , let's tokenize the description to have all the words. The 'description' column in the DataFrame 'processed_df' will contain tokenized versions of the text data from the original 'description' column.

In [ ]:
processed_df['description']= processed_df.description.apply(lambda row: word_tokenize(row))
processed_df.description.head()

0    [7, million, peopl, estim, die, everi, year, d...
1    [analyt, toxicolog, msc, uniqu, studi, cours, ...
2    [comput, scienc, foundat, mani, excit, cutting...
3    [polit, uncertainti, financ, recruit, demands,...
4    [thi, master, focus, linguist, research, help,...
Name: description, dtype: object

In [ ]:
vocabulary = set()
processed_df.description.apply(lambda row: [vocabulary.add(word) for word in row])

0       [None, None, None, None, None, None, None, Non...
1       [None, None, None, None, None, None, None, Non...
2       [None, None, None, None, None, None, None, Non...
3       [None, None, None, None, None, None, None, Non...
4       [None, None, None, None, None, None, None, Non...
                              ...                        
5974    [None, None, None, None, None, None, None, Non...
5975    [None, None, None, None, None, None, None, Non...
5976    [None, None, None, None, None, None, None, Non...
5977    [None, None, None, None, None, None, None, Non...
5978    [None, None, None, None, None, None, None, Non...
Name: description, Length: 5979, dtype: object

#### Now the set vocabulary contains all the words. In fact:

In [ ]:
vocabulary

{'mesc',
 'knowledge-bas',
 'dilemma',
 'engaged',
 'relationships',
 'two',
 'mini',
 'agro-',
 'breathe',
 'spear',
 'web-presence',
 'reorganis',
 'antique',
 'webst',
 'neurosensori',
 'nuanc',
 'deliverers',
 'statist',
 'high-tech',
 'dig',
 'stonehaven',
 'prototypes',
 'oncologist',
 'bankers',
 '96',
 'psychotherapist',
 'sharper',
 'inter-relationship',
 'non-democrat',
 'telecommunications',
 'characterized',
 'economists',
 'zooarchaeolog',
 'yet',
 'increases',
 'supporters',
 'nanoscale',
 'creator',
 'analyz',
 'crédit',
 'australian',
 'stakeholders.y',
 'ports',
 'consequ',
 'auf',
 'underli',
 'helmholtz-cent',
 'forensic/investig',
 'cables',
 'strathclyde',
 'inter-government',
 'bioscience',
 'twentieth',
 'superb',
 'appli',
 'project-work',
 'well-publicis',
 'commercialization',
 'got',
 'mainli',
 'conducted',
 'effect',
 'established',
 'alien',
 'however',
 'kick-start',
 'glastonbury',
 'signific',
 'thinking',
 'administrative',
 'decarbonisation',
 'comple

 #### Now we can assign unique ID to each term in the vocabulary. We create a df called vocabulary_df with 2 columns: Word and Id

In [ ]:
vocabulary_list = list(vocabulary)
vocabulary_dict = {word: index for index, word in enumerate(vocabulary_list)}
vocabulary_df=pd.DataFrame(list(vocabulary_dict.items()), columns=['Word', 'Id'])
print(vocabulary_df.head())

            Word  Id
0           mesc   0
1  knowledge-bas   1
2        dilemma   2
3        engaged   3
4  relationships   4


#### Now we can save it , we choosed to save it both in a json and in a csv

In [ ]:
vocabulary_df.to_csv('vocabulary.csv', index=False, header=False)
#remove header to keep the name of the columns

In [ ]:
# Store in a json llike a pandas dataframe
vocabulary_df.to_json('vocabulary.json', orient='records')

In [ ]:
# Write the dictionary to the JSON file directly
with open('vocabulary.json', 'w') as jsonfile:
    json.dump(vocabulary_dict, jsonfile)

#### Now let's create another dataframe called 'vocabulary_reverse' which, in addition to the columns from 'vocabulary', includes an extra column named 'reverse', containing the list of documents where that word is present

In [ ]:
vocabulary_reverse = vocabulary_df.copy()
print(vocabulary_reverse.head())

            Word  Id
0           mesc   0
1  knowledge-bas   1
2        dilemma   2
3        engaged   3
4  relationships   4


In [ ]:
tqdm.pandas()
vocabulary_reverse['reverse'] = vocabulary_reverse.Word.progress_apply(lambda item: list(processed_df.loc[processed_df.description.apply(lambda row: item in row)].index))

100%|██████████| 12545/12545 [01:49<00:00, 114.45it/s]


In [ ]:
vocabulary_reverse.head()

,Word,Id,reverse
0,mesc,0,[2131]
1,knowledge-bas,1,"[943, 4509, 4788, 4813, 5615]"
2,dilemma,2,"[181, 552, 1998, 3728, 4275, 5368]"
3,engaged,3,"[945, 3476]"
4,relationships,4,"[233, 437, 720, 1373, 1452, 1456, 1577, 2126, ..."


In [ ]:
vocabulary_reverse[vocabulary_reverse['reverse'].apply(lambda x: len(x) == 0)]

,Word,Id,reverse


#### But Inverted Index has to be a dictionary so:

In [ ]:
inverted_index = vocabulary_reverse.set_index('Id')['reverse'].to_dict()
count = 0
for key, value in inverted_index.items():
    if count < 5:
        print(f"Key: {key}, Value: {value}")
        count += 1
    else:
        break

Key: 0, Value: [2131]
Key: 1, Value: [943, 4509, 4788, 4813, 5615]
Key: 2, Value: [181, 552, 1998, 3728, 4275, 5368]
Key: 3, Value: [945, 3476]
Key: 4, Value: [233, 437, 720, 1373, 1452, 1456, 1577, 2126, 2764, 3004, 3903, 4376, 4386, 5526, 5641]


#### Since you do not want to compute the inverted index every time you use the Search Engine, it is worth thinking about storing it in a separate file and loading it in memory when needed.

In [ ]:
with open('inverted_index.json', 'w') as file:
    json.dump(inverted_index, file)

## [2.1.2] Execute the query

 We return documents which should contain all the words in the query

#### Selecting only the rows we need to return as output

In [ ]:
df_query = df[['courseName','universityName','description','url']].copy()
df_query.head()

,courseName,universityName,description,url
0,Air Quality Solutions - MSc,University of Leeds,Up to 7 million people are estimated to die ev...,https://www.findamasters.com/masters-degrees/c...
1,Analytical Toxicology MSc,King’s College London,The Analytical Toxicology MSc is a unique stud...,https://www.findamasters.com/masters-degrees/c...
2,Applied Computer Science and Artificial Inte...,University of Bradford,Computer science is the foundation of many exc...,https://www.findamasters.com/masters-degrees/c...
3,Applied Economics (Banking and Financial Mar...,University of Bath,From political uncertainty to finance and recr...,https://www.findamasters.com/masters-degrees/c...
4,Applied Linguistics - MSc,University of Glasgow,This Masters focuses on how linguistic researc...,https://www.findamasters.com/masters-degrees/c...


#### extracting documents for a specific query using the function engine

In [ ]:
custom_functions.engine('advance knowledge')

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,administration,country,url
1,Analytical Toxicology MSc,King’s College London,Faculty of Life Sciences & Medicine,Full time,The Analytical Toxicology MSc is a unique stud...,See Course,Please see the university website for further ...,MSc,Full-time: One year,London,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
5,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,Businesses and governments rely on sound finan...,September,"UK: £18,000 (Total) \nInternational: £34,750 (...",MSc,1 year full time,Leeds,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
8,Addictions MSc,King’s College London,"Institute of Psychiatry, Psychology and Neuros...",Full time,Join us for an online session for prospective ...,September,Please see the university website for further ...,MSc,One year FT,London,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2056,Engineering with Management - MSc,University of Bristol,Faculty of Engineering,Full time,The MSc in Engineering with Management is desi...,September,Please see the university website for further ...,MSc,1 year full-time,Bristol,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2069,Enhanced Radiotherapy and Oncology Practice ...,Sheffield Hallam University,Postgraduate Courses,Part time,Enhance your knowledge of specific radiotherap...,"September, January",For part-time study the fee will be calculated...,MSc,3 years part time distance learning,Sheffield,Online,United Kingdom,https://www.findamasters.com/masters-degrees/c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,Engineering Management (MSc),University of Gloucestershire,School of Computing and Engineering,Full time,This course will equip you with the necessary ...,September,Please see the university website for further ...,MSc,See website for details,Gloucester,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2032,Engineering Management MSc,University of Bradford,Faculty of Engineering & Digital Technologies,Full time,The MSc Engineering Management programme will ...,"September, January",Please see the university website for further ...,MSc,1 year Full time,Bradford,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2034,Engineering Management MSc,Middlesex University,Faculty of Science and Technology,Full time,The career of an engineering project manager i...,October,"UK students \nFull-time students: £10,500 \nPa...",MSc,1 year full-time; 2 years part-time,London,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
4083,M2 Research in Management and Innovation,Université Côte d’Azur,UCA International MSc Program,Full time,"Our modern industry structures, value chains a...",September,The tuition currently stands at €243 as well a...,MSc,2 years,Nice,On Campus,France,https://www.findamasters.com/masters-degrees/c...


## [2.2] Conjunctive query & Ranking score

## [2.2.1] Inverted index

### For each word, you want the list of documents in which it is contained and the relative tfIdf score.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#### The following code computes TF-IDF values for the text in the 'description' column and stores the TF-IDF matrix in the DataFrame tfidf_data, where each row corresponds to a document, each column corresponds to a word, and the values represent TF-IDF scores.

In [ ]:
tfidf = TfidfVectorizer(input='content', lowercase=False, tokenizer=lambda text: text) # , max_df=0.1
results = tfidf.fit_transform(processed_df['description'])
result_dense = results.todense()
tfidf_data = pd.DataFrame(result_dense.tolist(), index=processed_df.index, columns=tfidf.get_feature_names_out())
print(tfidf_data.head())

   'all  'appli  'bench  'big  'chip  'credits   'd  'disability  'educ  \
0   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   
1   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   
2   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   
3   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   
4   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   

   'entri  ...  ﻿over  ﻿prove  ﻿regist  ﻿th  ﻿the  ﻿theoret  ﻿thi  ﻿want   ﻿y  \
0     0.0  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0   
1     0.0  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0   
2     0.0  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0   
3     0.0  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0   
4     0.0  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0   

    ﻿﻿  
0  0.0  
1  0.0  
2  0.0  
3  0.0  
4  0.0  

[5 rows

#### Creating a dictionary in the format Word:[(document1, tfIdf_{term,document1}), (document2, tfIdf_{term,document2})...

In [ ]:
inverted_index_2 = {}
for word in tfidf_data.columns:
    word_docs = tfidf_data.index[tfidf_data[word] > 0].tolist()
    word_doc_scores = [(doc, tfidf_data.loc[doc, word]) for doc in word_docs]
    inverted_index_2[word] = word_doc_scores
count = 0
for word, doc_scores in inverted_index_2.items():
    print(f"Word: {word}")
    print("Document Scores:")
    print(doc_scores)
    count += 1
    if count == 10:
        break

Word: 'all
Document Scores:
[(3871, 0.1849258796135223)]
Word: 'appli
Document Scores:
[(2325, 0.1722113554646612)]
Word: 'bench
Document Scores:
[(333, 0.3466489203597607)]
Word: 'big
Document Scores:
[(1217, 0.2404462962515215), (1225, 0.1886881267487287), (1290, 0.2107926430819876), (5692, 0.17695012088105525)]
Word: 'chip
Document Scores:
[(1902, 0.1992739368962018)]
Word: 'credits
Document Scores:
[(2317, 0.18303193893688977), (5926, 0.08382342995897132)]
Word: 'd
Document Scores:
[(2160, 0.194367236072988), (4863, 0.14435725548964343)]
Word: 'disability
Document Scores:
[(4783, 0.17222181095019545)]
Word: 'educ
Document Scores:
[(2618, 0.2647135148766833)]
Word: 'entri
Document Scores:
[(641, 0.20952396586542038)]


#### But we need to have the word's ID as the key, not the word itself, we can take this ID from the vocabulary_df. We'll create a new dictionary that meets the requirements specified in the prompt.

In [ ]:
new_inverted_index = {}
for word, doc_scores in inverted_index_2.items():
    word_id = vocabulary_df[vocabulary_df['Word'] == word]['Id'].values
    if len(word_id) > 0:
        word_id = word_id[0]
        new_inverted_index[word_id] = doc_scores

#### Let's see the result

In [ ]:
n = 10
partial_inverted_index = {k: new_inverted_index[k] for k in list(new_inverted_index.keys())[:n]}
print(partial_inverted_index)

{7237: [(3871, 0.1849258796135223)], 8785: [(2325, 0.1722113554646612)], 9700: [(333, 0.3466489203597607)], 3365: [(1217, 0.2404462962515215), (1225, 0.1886881267487287), (1290, 0.2107926430819876), (5692, 0.17695012088105525)], 1201: [(1902, 0.1992739368962018)], 650: [(2317, 0.18303193893688977), (5926, 0.08382342995897132)], 11512: [(2160, 0.194367236072988), (4863, 0.14435725548964343)], 7980: [(4783, 0.17222181095019545)], 2565: [(2618, 0.2647135148766833)], 4168: [(641, 0.20952396586542038)]}


#### Compare document with cosine similarity

Given a query we will compare the query and evaluate the cosine similarity between th query and the extracted document through the engine function. We retrieve k = 10 documents: the first k ordered by cosine similarity.

In [ ]:
df_query_processed= processed_df[['courseName','universityName','description','url']].copy()

In [ ]:
df_query_processed.head()

,courseName,universityName,description,url
0,air quality solutions msc,university leeds,"[7, million, peopl, estim, die, everi, year, d...",https //www.findamasters.com/masters-degrees/c...
1,analytical toxicology msc,king college london,"[analyt, toxicolog, msc, uniqu, studi, cours, ...",https //www.findamasters.com/masters-degrees/c...
2,applied computer science artificial intelligen...,university bradford,"[comput, scienc, foundat, mani, excit, cutting...",https //www.findamasters.com/masters-degrees/c...
3,applied economics banking financial markets on...,university bath,"[polit, uncertainti, financ, recruit, demands,...",https //www.findamasters.com/masters-degrees/c...
4,applied linguistics msc,university glasgow,"[thi, master, focus, linguist, research, help,...",https //www.findamasters.com/masters-degrees/c...


In [ ]:
tfidf = TfidfVectorizer(input='content', lowercase=False, tokenizer=lambda text: text) # , max_df=0.1
results = tfidf.fit_transform(df_query_processed['description'])
result_dense = results.todense()
tfidf_data = pd.DataFrame(result_dense.tolist(), index=df_query_processed.index, columns=tfidf.get_feature_names_out())

I need to take the cosine similary for the words in the query.

In [ ]:
def rank_documents(query1):
    """
    Given a query, computes the tfidf for that query and evaluate the cosine similarity con for query given
    the document extracted with engine function. Returns the top-k documents
    """
    ## COMPUTING THE TFIDF FOR THE QUERY

   # Tokenize the query and save the stammed words for computinf tfidf
    tokens = word_tokenize(query1)
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    tfidf_vectorizer = TfidfVectorizer()

    # Combine the stemmed tokens considering it single string (document)
    stemmed_query = ' '.join(stemmed_tokens)

    # Fit the vectorizer on the list of queries and transform the query into a tfidf vector
    query_tfidf = tfidf_vectorizer.fit_transform([stemmed_query])

    # Convert the TF-IDF matrix to a dense pandas DataFrame
    tfidf_query = pd.DataFrame(query_tfidf.todense(), index=['query'], columns=tfidf_vectorizer.get_feature_names_out())

    ## OBTAINE THE TFIDF BEWTEEN THE QUERY AND THE DOCUMENT
    # Initialize a heap for the k-top results
    k_top_results = []

    # Iterate over the documents
    for idx, document_text in enumerate(functions.engine(query1)['description']):

        for index, tfidf in inverted_index_2['air']:
            if index in functions.engine('air pollution')['description'].index:
                print('nel documento',{index}, 'la cosine similarity e',tfidf)
        # Compute cosine similarity between the query and the document
        similarity_score = cosine_similarity(tfidf_query, tfidf_document)[0, 0]

       ## USING A HEAP TO HAVE THE TOP-K DOCUMENTS
        # Append the similarity score and document index to the k_top_results list
        k_top_results.append((round(similarity_score,5), functions.engine(query1)['description'].index[idx]))


    # Retrieve the k-top results from the heap using nlargest
    k_top_results = heapq.nlargest(10, k_top_results, key=lambda x: x[0])
    rank_df = pd.DataFrame()

    for cossim, indexsorted in k_top_results:
        # Return the information about the document sorted by cosine similarity
            document_info = df_query.loc[[indexsorted]].copy()
#            return document_info
            document_info['Similarity'] = cossim
            rank_df = pd.concat([rank_df, document_info])

    # Eventually we could reset the index of the final DataFrame
    # rank_df = rank_df.reset_index(drop=True)

    return rank_df

Copules of `EXAMPLES`

In [ ]:
functions.rank_documents("advanced knowledge")

,courseName,universityName,description,url,Similarity
1,Analytical Toxicology MSc,King’s College London,The Analytical Toxicology MSc is a unique stud...,https://www.findamasters.com/masters-degrees/c...,1.0
8,Addictions MSc,King’s College London,Join us for an online session for prospective ...,https://www.findamasters.com/masters-degrees/c...,1.0
2056,Engineering with Management - MSc,University of Bristol,The MSc in Engineering with Management is desi...,https://www.findamasters.com/masters-degrees/c...,1.0
2073,Enterprise Management MSc/PGDip,University College Birmingham,In today’s increasingly competitive business e...,https://www.findamasters.com/masters-degrees/c...,1.0
2089,Entrepreneurship and Innovation Management MSc,University of Exeter,Overview\nGain a fully transferable and implem...,https://www.findamasters.com/masters-degrees/c...,1.0
2044,Engineering Materials (Advanced Mechanical E...,University of Southampton,Develop specialist knowledge of mechanical eng...,https://www.findamasters.com/masters-degrees/c...,1.0
4169,Accounting and Finance - MSc,University of Sussex,On this MSc you’ll advance your accounting and...,https://www.findamasters.com/masters-degrees/c...,1.0
4180,Management and Digital Business (with Advanc...,Liverpool John Moores University,This Advanced Practice course provides an in-d...,https://www.findamasters.com/masters-degrees/c...,1.0
4203,Management in Construction (MSc),Kingston University,This industry-accredited course is suitable fo...,https://www.findamasters.com/masters-degrees/c...,1.0
115,Bioscience (MSc),KAUST,The Bioscience (B) program plays a key role in...,https://www.findamasters.com/masters-degrees/c...,1.0


In [ ]:
functions.rank_documents("air pollution")

,courseName,universityName,description,url,Similarity
5430,Air Pollution Management and Control - MSc/P...,University of Birmingham,Our Air Pollution Management and Control MSc i...,https://www.findamasters.com/masters-degrees/c...,0.99589
0,Air Quality Solutions - MSc,University of Leeds,Up to 7 million people are estimated to die ev...,https://www.findamasters.com/masters-degrees/c...,0.98058
2128,Environmental Analytical Chemistry - MSc,University of Bristol,"Climate change, and air, water and soil pollut...",https://www.findamasters.com/masters-degrees/c...,0.98058
2150,Environmental Engineering - MSc,Imperial College London,This course provides training in all aspects o...,https://www.findamasters.com/masters-degrees/c...,0.94868
2216,Environmental Pollution Control MSc,University of Southampton,Explore Environmental Pollution Control at a g...,https://www.findamasters.com/masters-degrees/c...,0.83205


#[7] Algorithmic question

The problem involves helping Leonardo create a fake report on the number of hours he worked each day for the past d days. Leonardo has the total sum of hours he worked, the number of days, and HR limitations for each day.

## Implementation of the code

In [ ]:
# Input the total number of days and the total number of hours
total_days, sum_hours = map(int, input().split())

# Initialize a list to store the ranges of hours for each day
day_min_max = []

# Input the minimum and maximum hours for each day and create a list of ranges
for _ in range(total_days):
    min1, max1 = map(int, input().split())
    day_min_max.append([i for i in range(min1, max1 + 1)])

def find_combinations_with_sum(nested_list, target_sum):
    """
    Having a nested list with the information of the overall amount of days and hour
    worked and informationa bout mintime e maxtime for a single day, returns wether
    it it possibile to return a fake report which respect costrains
    """
    # Get the minimum and maximum hours for each day
    min_hours = [day[0] for day in day_min_max]
    max_hours = [day[-1] for day in day_min_max]

    # Check if the total hours are within the valid range
    if sum_hours >= sum(min_hours) and sum_hours <= sum(max_hours):
        print('YES')
    else:
        print('NO')

    # Iterate through all combinations of hours and print those with the target sum
    for combination in product(*nested_list):
        if sum(combination) == target_sum:
            print(*combination)

# Call the function with the provided day_min_max and sum_hours
find_combinations_with_sum(day_min_max, sum_hours)


1 1
2 3
NO



## Let's combine the time complexity of individual parts to determine the overall time complexity of the provided code:



Initializing min_hours and max_hours: O(total_days)
* Checking total hours range: O(1)
* Generating combinations: O(product of sizes of nested_list)
* Overall, the dominant factor is the third part involving the generation of combinations. Therefore, the overall time complexity is O(product of sizes of nested_list).

#### example : if we have only two working days


let's calculate the number of combinations generated by the product function for two days, each with a range of (max - min + 1) hours

day1 = max_hour - min_hour + 1 = n1

day2 = max_hour - min_hour + 1 = n2


Complexity would be : O(n1 * n2 )

## ChatGpt Implementation



both awsers are right but we think the complexity of the code that Chatgpt has been generated is better since it use backtracking algorithm which avoids generation all combination

##  **Optimality**:


The solution can be considered suboptimal in terms of time complexity, especially when the ranges for each day are large.
It generates all combinations using itertools.product, which can be inefficient for large inputs.
An improvement could be to use a backtracking algorithm that explores valid combinations more efficiently, terminating the search as soon as a valid combination is found.